In [1]:
import os
import warnings
import numpy as np
import luigi
import emu.neuralynx_io as nlx
from tqdm import tqdm as tqdm
from emu.pdil.raw import Electrophysiology,Participant,get_data_manifest,points_to_choice
from emu.neuralynx_io import nev_as_records
from emu.nwb import nlx_to_nwb
from pynwb import TimeSeries, NWBFile,NWBHDF5IO
from pynwb.ecephys import ElectricalSeries
from pynwb.misc import AnnotationSeries
import pandas as pd
import datetime
import glob

In [2]:
all_files = get_data_manifest()

In [3]:
seeg_root = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/')

nev_path = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.Events.nev')
ncs_paths = sorted(glob.glob(os.path.join(seeg_root,'PO_Day_04.*.ncs')))

# ncs = nlx.load_ncs()
print('nev path: ',nev_path)
print('ncs_paths:')
for p in ncs_paths[:10]:
    print(p)

nev path:  /home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.Events.nev
ncs_paths:
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC1.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC10.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC100.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC101.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC102.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC103.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC104.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC105.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC106.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC107.ncs


In [4]:
p = Participant(patient_id=1,raw_files=all_files,seeg_raw_path=seeg_root)

/usr/local/lib/python3.6/dist-packages/luigi/parameter.py:283: UserWarning: Parameter "file_path" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))


In [5]:
# Create a list of download tasks for the POD4 ncs files
all_ncs_tasks = list(p.cache_ncs())
d4_ncs_tasks = [t for t in all_ncs_tasks if 'PO_Day_04' in t.output().path]
print([t.file_name for t in d4_ncs_tasks[:10]],'\n')


# Create a list of download tasks for the POD4 nev files
d4_nev = [t for t in list(p.cache_nev()) if 'PO_Day_04.Events.nev' in t.output().path][0]
print(d4_nev)

['PO_Day_04.CSC1.ncs', 'PO_Day_04.CSC10.ncs', 'PO_Day_04.CSC100.ncs', 'PO_Day_04.CSC101.ncs', 'PO_Day_04.CSC102.ncs', 'PO_Day_04.CSC103.ncs', 'PO_Day_04.CSC104.ncs', 'PO_Day_04.CSC105.ncs', 'PO_Day_04.CSC106.ncs', 'PO_Day_04.CSC107.ncs'] 

NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.Events.nev)


In [6]:
# Make sure the files are cached locally
luigi.build(d4_ncs_tasks+[d4_nev],workers=4)

DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC1.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004419788_PO_Day_04_CSC1_n_5df6429b44   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC10.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004417388_PO_Day_04_CSC10__70601f89a3   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC100.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632881825314_PO_Day_04_CSC100_b7068f5484   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC101.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632882696587_PO_Day_04_CSC101_2250a7b1d0   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC102.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e

DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC134.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632882574035_PO_Day_04_CSC134_8886416c0a   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC135.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632883516344_PO_Day_04_CSC135_99d1b2bec3   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC136.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632864772461_PO_Day_04_CSC136_e064642e75   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC137.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632860151053_PO_Day_04_CSC137_cb21834ca7   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC138.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc

DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC17.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004386219_PO_Day_04_CSC17__e8f5f5b5c0   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC170.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632888652715_PO_Day_04_CSC170_f3b099865e   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC171.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632873647364_PO_Day_04_CSC171_7f8307f74f   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC172.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632859386842_PO_Day_04_CSC172_3610edab25   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC173.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc_

DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC43.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_631983694728_PO_Day_04_CSC43__2e24d53044   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC44.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_631983667128_PO_Day_04_CSC44__f20b8e6014   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC45.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_631983675528_PO_Day_04_CSC45__8ddf8a93d5   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC46.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_631983700728_PO_Day_04_CSC46__e962359e80   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC47.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_6

DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC79.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632860094602_PO_Day_04_CSC79__2c0cceb0c7   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC8.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004585821_PO_Day_04_CSC8_n_18d06b43a2   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC80.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632882524947_PO_Day_04_CSC80__db0a8c8bc2   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC81.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632865054663_PO_Day_04_CSC81__944a834a66   has status   DONE
DEBUG: Checking if NLXRaw(file=/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC82.ncs) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_63

True

In [7]:
nwb = p.create_nwb(nev_path,ncs_paths,blocks=[3,4,5,6],desc='Patient 1 | Post-op Day 4')

compressing channels:  43%|████▎     | 77/178 [01:21<01:46,  1.06s/it]

Error loading PO_Day_04.CSC17.ncs
skipping...


compressing channels: 100%|██████████| 178/178 [03:07<00:00,  1.05s/it]


Failed to load:  wire_14_electrode_9
Failed to load:  wire_15_electrode_8
0 missing tasks
0 missing tasks


In [8]:
nwb

root pynwb.file.NWBFile at 0x140086567722232
Fields:
  acquisition: {
    channel_0 <class 'pynwb.base.TimeSeries'>,
    ttl <class 'pynwb.misc.AnnotationSeries'>,
    wire_10_electrode_1 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_10 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_2 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_3 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_4 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_5 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_6 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_7 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_8 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_10_electrode_9 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_11_electrode_1 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_11_electrode_10 <class 'pynwb.ecephys.ElectricalSeries'>,
    wire_11_electrode_11 <class 'pynwb.ece

In [9]:
nwb.acquisition['ttl']

ttl pynwb.misc.AnnotationSeries at 0x140090293191624
Fields:
  comments: no comments
  conversion: 1.0
  data: ['block_start' 'block_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'block_start' 'block_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'block_start'
 'block_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'block_start' 'block_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_s

In [10]:
nwb.trials.to_dataframe().head()

,start_time,stop_time,outcome
id,,,
0,208.704119,225.619469,C-C
1,225.637112,240.257794,C-C
2,240.270138,254.523553,D-C
3,254.536538,268.636368,C-D
4,268.652939,303.003493,D-C


In [11]:
with NWBHDF5IO(os.path.join(seeg_root,'..','processed','PO_Day_04.nwb'),'w') as io:
    io.write(nwb)

/usr/local/lib/python3.6/dist-packages/hdmf/build/map.py:949: MissingRequiredWarning: attribute 'unit' for 'data' (TimeSeries)
  warnings.warn(msg, MissingRequiredWarning)


In [15]:
from emu.luigi.box import BoxClient,path_to_obj,file_id_to_path

In [13]:
box = BoxClient()

In [17]:
file_id_to_path(633031167652,box.conn)

'/EMU/STUDY_PDil/PT_01/SEEG/electrode_locations.csv'

In [19]:
src_path = os.path.join(seeg_root,'..','processed','PO_Day_04.nwb')

box.upload('/EMU/STUDY_PDil/PT_01/SEEG/processed',src_path)

<Box File - 632886996988 (PO_Day_04.nwb)>